# Plotting with *pymovements*

*Pymovements* provides a collection of built-in plotting functions 
to visualize gaze data in both temporal and spatial dimensions.

These functions make it easy to explore and present your data, 
from individual trial visualizations to aggregated participant-level analyses.

In this tutorial, you’ll learn how to:

- load and prepare a sample dataset for plotting
- compute the necessary properties
- plot gaze traces over time (traceplot)
- visualize fixations and saccades on a stimulus (scanpathplot)
- generate heatmaps showing gaze density (heatmap)
- plot the saccadic main sequence (main-sequence)
- style the plot

All examples use the small and reproducible :doc:`ToyDataset <../../datasets/ToyDataset>` 
that comes with pymovements.

## Loading and preprocessing data

In [ ]:
import pymovements as pm

dataset = pm.Dataset('ToyDataset', path='data/ToyDataset')
dataset.download()
dataset.load()

# convert the raw x and y coordinates in pixels to degrees in visual angle
dataset.pix2deg()

# convert these positions into velocitites
dataset.pos2vel()

## Making the Traceplot

A traceplot visualizes the gaze path over time, showing the sequence of fixations and saccades across a stimulus. It represents eye movements as a continuous trajectory connecting gaze points, where each point corresponds to a specific position and moment in time. This helps reveal viewing patterns — where a person looks, for how long, and in what order.

Traceplots are useful for:
- Verifying that gaze data have been parsed and aligned correctly.
- Exploring viewing behavior across conditions or participants.
- Identifying artifacts or data quality issues.

A basic traceplot can be created with only a `Gaze` object:

In [ ]:
# detect saccades
dataset.detect_events('microsaccades')
# compute amplitude and peak velocity of the detected saccades
dataset.compute_event_properties(['amplitude', 'peak_velocity'])

In [ ]:
pm.plotting.traceplot(dataset.gaze[0])

## Creating the Scanpath Plot

A scanpath plot illustrates the sequence of eye movements while viewing a stimulus, depicting the order of fixations and saccades. Fixations are shown as circles whose sizes reflect their duration, while lines connect them to trace the gaze path.

In [ ]:
# detect fixations using the I-DT algorithm
dataset.detect('idt', minimum_duration=100, dispersion_threshold=1.0)
# compute event properties, such as the mean position of each fixation
dataset.compute_event_properties(("location", {"position_column": "pixel"}))

In [ ]:
pm.plotting.scanpathplot(dataset.gaze[3])

In [ ]:
pm.plotting.scanpathplot(dataset.gaze[3], add_traceplot=True)

## Heatmap Plotting

The heatmap shows the distribution of gaze positions across the experiment screen, with color values indicating the time spent at each position in seconds.

Let's create a heatmap using the `heatmap` function from the `pymovements` library. We will use the default `gridsize` of 10x10 with interpolation and display the colorbar.

In [ ]:
pm.plotting.heatmap(dataset.gaze[2])

In [ ]:
# you can customize various aspects of the heatmap plot, such as the grid
# size, color map, and labels.

figure = pm.plotting.heatmap(
    gaze=dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation On',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
)

To better understand the effect of the `gridsize` parameter on the heatmap, we can turn off the interpolation. By doing this, we can clearly visualize the individual bins used to calculate the heatmap. With interpolation turned off, the heatmap will display the raw bin values rather than a smoothed representation.

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Interpolation Off',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[10, 10],
    interpolation='none'
)

Increasing the `gridsize` parameter results in a finer grid and more detailed heatmap representation. With a higher grid size, we divide the plot into smaller bins, which can capture more nuances in the data distribution

In [ ]:
figure = pm.plotting.heatmap(
    dataset.gaze[5],
    position_column='pixel',
    origin='upper',
    show_cbar=True,
    cbar_label='Time [s]',
    title='Gaze Heatmap with Higher Grid Size',
    xlabel='X [pix]',
    ylabel='Y [pix]',
    gridsize=[25, 25]
)

## Plotting the Saccadic Main Sequence

Pass the event dataframe to the plotting function

In [ ]:
# show the first three event dataframes.
# note that you can adjust the styling of the plot, e.g. setting a low
# alpha value allows you to see overlapping data points
for event_df in dataset.events[:3]:
    pm.plotting.main_sequence_plot(
        event_df,
        title='Main sequence plot for '
        f'text {event_df[0, "text_id"]}, '
        f'page {event_df[0, "page_id"]}',
        alpha=0.5,
        color='green',
        marker='x',
        marker_size=30,
    )